## **GESTIÓN DE DATOS**

 **Nombre:** Juliana Castañeda Arredondo

### ***PROBLEMA 3***

##### **I) Diseña una clase que modele la estructura de un banco con diferentes tipos de cuentas bancarias como se muestra a continuación:**


**Clase Banco:** Contiene una lista de clientes, donde cada cliente tiene una cuenta bancaria.

**Clase Cliente:** Cada cliente tiene un nombre y una cuenta.

**Clase Cuenta:** Tiene atributos como número de cuenta y saldo.

In [173]:
# Librerías básicas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

##### **II) Añade un método en la clase Banco que permita transferir dinero entre dos cuentas de clientes diferentes. Si la cuenta origen es una Cuenta Corriente y se produce un sobregiro, se debe aplicar la penalización automáticamente.**

In [174]:
# Primero debemos definir la clase la cual poseen los datos de los clientes


class Clientes:
    def __init__(self, numero_cuenta, nombre, tipo_cuenta, saldo=0):   # Se tiene que aclarar que el saldo inicia en cero

        # Agregamos los atributos
        self.numero_cuenta = numero_cuenta       # Número de cuenta del cliente (0001,0002,...)
        self.nombre = nombre                     # Nombre del cliente
        self.tipo_cuenta = tipo_cuenta.tipo      # Tipo de cuenta: Cuenta Corriente o Cuenta de Ahorros
        self.saldo = saldo                       # Saldo de la cuenta
        
        self.movimientos = []                    # Lo marcamos para después imprimir los movimientos de los clientes (retirar o depositar)
                                                 # es una lista vacía.

    


    # -------------------- DEPÓSITOS A OTRAS CUENTAS --------------------


    # Se verifica si existe un receptor ya registrado en nuestra base de datos
    def depositar(self, monto, receptor_numero_cuenta, clientes):
        receptor = next((cliente for cliente in clientes if cliente.numero_cuenta == receptor_numero_cuenta), None)
    
        if receptor is None:
            print(f"ERROR: El receptor con número de cuenta {receptor_numero_cuenta} no existe en la base de datos.")
            return

        self.saldo -= monto                         # El saldo del emisor se reduce 
        receptor.saldo += monto                     # El saldo del receptor incrementa 
    


        # EMISOR
        movimiento_emisor = {
            'Fecha': datetime.now(),                     # Registramos la fecha en la que se realizaron movimientos para después registrarlos
            'Cliente': self.nombre,
            'Tipo de Cuenta': self.tipo_cuenta,          # Cuenta Corriente o Cuenta de Ahorros      
            'Saldo': self.saldo,                         # Saldo final tras hacer el depósito a otra cuenta         
            'Tipo de Movimiento': 'Deposito a tercero',
            'Monto': monto,                              # Monto depositado/transferido
            'Receptor': receptor.nombre
        }
        self.movimientos.append(movimiento_emisor)       # Con append vamos modificando la tabla de "movimientos" del emisor
    
        # RECEPTOR
        movimiento_receptor = {
            'Fecha': datetime.now(),
            'Cliente': receptor.nombre,
            'Tipo de Cuenta': receptor.tipo_cuenta,
            'Saldo': receptor.saldo,
            'Tipo de Movimiento': 'Deposito recibido',
            'Monto': monto,
            'Emisor': self.nombre
        }
        receptor.movimientos.append(movimiento_receptor)  # Con append vamos modificando la tabla de "movimientos" del receptor

    


    # -------------------- RETIROS --------------------
    
    # Definimos como se van a realizar los retiros de efectivo
    def retirar(self, monto):


       
        # ---------- CUENTA CORRIENTE ------------

        if self.tipo_cuenta == 'Corriente':
            saldo_resultante = self.saldo - monto
            if saldo_resultante >= -1000:                           # Determinamos que el saldo negativo debe ser de mil pesos 
                self.saldo -= monto                                 # El saldo disminuye al momento de retirar efectivo
                if self.saldo < 0:
                    penalizacion = abs(self.saldo) * 0.07           # Si el saldo es negativo, se aplica una penalización del 7% del monto retirado
                    self.saldo -= penalizacion
                    tipo_movimiento = 'Retiro con penalizacion'
                else:
                    tipo_movimiento = 'Retiro'                      # Si el saldo no tiene penalización, es retiro normal
                movimiento = {
                    'Fecha': datetime.now(),
                    'Cliente': self.nombre,
                    'Tipo de Cuenta': self.tipo_cuenta,
                    'Saldo': self.saldo,
                    'Tipo de Movimiento': tipo_movimiento,
                    'Monto': monto
                }
                self.movimientos.append(movimiento)                 # Con append vamos modificando la tabla de "movimientos"
            else:
                print("ERROR: No puede retirar dinero. El saldo no puede ser menor que menos mil pesos (-$1000).")



        # ---------- CUENTA DE AHORRO ------------

        else:  
            if monto <= self.saldo:
                self.saldo -= monto                                # El saldo disminuye al momento de retirar efectivo
                tipo_movimiento = 'Retiro'
            else:
                print("ERROR: Fondos insuficientes en una cuenta de ahorros.")
                return

        movimiento = {
            'Fecha': datetime.now(),
            'Cliente': self.nombre,
            'Tipo de Cuenta': self.tipo_cuenta,
            'Saldo': self.saldo,
            'Tipo de Movimiento': tipo_movimiento,
            'Monto': monto
        }
        self.movimientos.append(movimiento)                       # Con append vamos modificando la tabla de "movimientos"

**Cuenta Corriente:** debe permitir retirar dinero, incluso si el saldo es menor que el monto
solicitado (permitiendo un saldo negativo), pero con una penalización del 7% sobre el monto
del sobregiro.

In [175]:
class CuentaCorriente:
    def __init__(self):
        self.tipo = 'Corriente'
        self.limite_sobregiro = -1000

cuenta_corriente = CuentaCorriente()

**NOTA:** El límite de saldo negativo para la ***cuenta corriente*** es de mil pesos (-$1000).

**Cuenta Ahorros:** No permite sobregiros, pero acumula intereses mensuales a una tasa fija.

In [176]:
class CuentaAhorros:
    def __init__(self):
        self.tipo = 'Ahorros'
        self.tasa_interes = 0.02

cuenta_ahorros = CuentaAhorros()

**NOTA:** La tasa de interés para la ***cuenta de ahorros*** es del 2%.

#### ***Características a implementar:***


**Cliente:** Debe contener atributos como nombre y su cuenta bancaria asociada (puede ser de
cualquiera de los dos tipos: Cuenta Corriente o Cuenta Ahorros).

In [177]:
# Realizamos la creación de un dataframe con la información de todos los clientes 

def crear_dataframe_clientes(clientes):
    data = {
        'Nombre': [cliente.nombre for cliente in clientes],
        'Tipo de Cuenta': [cliente.tipo_cuenta for cliente in clientes],
        'Numero de Cuenta': [cliente.numero_cuenta for cliente in clientes],
        'Saldo': [cliente.saldo for cliente in clientes],
        'Fecha de Transaccion': [datetime.now() for _ in clientes]
    }
    df = pd.DataFrame(data)
    return df

**Banco:** Debe poder agregar clientes (y sus cuentas) a la lista de clientes

In [178]:
# Realizamos un dataframe que nos permita añadir clientes

def agregar_cliente(cliente):
    nuevo_cliente = pd.DataFrame([{
        'Nombre': cliente.nombre,
        'Tipo de Cuenta': cliente.tipo_cuenta,
        'Numero de Cuenta': cliente.numero_cuenta,
        'Saldo': cliente.saldo,
        'Fecha de Transaccion': datetime.now()
    }])
    global df_clientes
    
    df_clientes = pd.concat([df_clientes, nuevo_cliente], ignore_index=True)

Tiene métodos como depositar(monto) y retirar(monto)

In [179]:
# Función para registrar los movimientos de todos los clientes
def registrar_movimientos(clientes):
    movimientos = []
    for cliente in clientes:
        movimientos.extend(cliente.movimientos)
    df_movimientos = pd.DataFrame(movimientos, columns=['Fecha', 'Cliente', 'Tipo de Cuenta', 'Tipo de Movimiento', 'Monto'])
    return df_movimientos

In [180]:
# Crear una lista inicial de clientes con número de cuenta y saldo inicial
clientes = [ ]

In [181]:
# Crear el DataFrame inicial de clientes
df_clientes = crear_dataframe_clientes(clientes)

In [182]:
# Registrar y mostrar los movimientos de todos los clientes
df_movimientos = registrar_movimientos(clientes)

### **APLICACIÓN DEL REGISTRO DE CLIENTES**

**REGISTRO DE CLIENTES**

Vamos ahora a registrar datos de varios clientes para ver su funcionamiento:

In [183]:
# ------------ REGISTRO DE CLIENTES ------------

# Agregamos dos clientes

cliente1 = Clientes("0001", "Juliana Castaneda", cuenta_ahorros, 500)
cliente2 = Clientes("0002", "Ana Laura Ruiz", cuenta_corriente, 1500)

clientes.append(cliente1)
clientes.append(cliente2)


# Actualizar y mostrar la información de los clientes y sus movimientos
df_clientes = crear_dataframe_clientes(clientes)
df_clientes                                         # Muestra información de los clientes antes de las transacciones

,Nombre,Tipo de Cuenta,Numero de Cuenta,Saldo,Fecha de Transaccion
0,Juliana Castaneda,Ahorros,0001,500,2024-12-26 23:20:30.848667
1,Ana Laura Ruiz,Corriente,0002,1500,2024-12-26 23:20:30.848667


Esto nos arroja los clientes registrados previamente donde tenemos los siguientes datos:

* Nombre
* Tipo de cuenta
* Número de cuenta
* Saldo actual del cliente
* Fecha de registro

**REGISTRO DE TRANSACCIONES**

Vamos ahora a realizar un **depósito**:

In [184]:
# ------------ DEPÓSITOS ------------

cliente1.depositar(200, "0002", clientes)  # Juliana Castañeda deposita $200 a Ana Laura Ruiz


# Actualizar DataFrame de clientes después del depósito

df_clientes = crear_dataframe_clientes(clientes)

df_movimientos = registrar_movimientos(clientes)
df_movimientos 

,Fecha,Cliente,Tipo de Cuenta,Tipo de Movimiento,Monto
0,2024-12-26 23:20:30.945597,Juliana Castaneda,Ahorros,Deposito a tercero,200
1,2024-12-26 23:20:30.945597,Ana Laura Ruiz,Corriente,Deposito recibido,200


La anterior tabla nos muestra la cantidad de movimientos realizados.

Ahora vamos a actualizar la base de datos de los clientes para ver cuánto saldo tiene cada uno.

In [185]:
df_clientes

,Nombre,Tipo de Cuenta,Numero de Cuenta,Saldo,Fecha de Transaccion
0,Juliana Castaneda,Ahorros,0001,300,2024-12-26 23:20:30.945597
1,Ana Laura Ruiz,Corriente,0002,1700,2024-12-26 23:20:30.945597


Vamos ahora a realizar un **retiro de efectivo**:

In [186]:
# ------------ RETIROS ------------

cliente1.retirar(100)                      # Juliana Castañeda retira $100

# Actualizar DataFrame de clientes después de las transacciones
df_clientes = crear_dataframe_clientes(clientes)

df_movimientos = registrar_movimientos(clientes)
df_movimientos 


,Fecha,Cliente,Tipo de Cuenta,Tipo de Movimiento,Monto
0,2024-12-26 23:20:30.945597,Juliana Castaneda,Ahorros,Deposito a tercero,200
1,2024-12-26 23:20:31.717333,Juliana Castaneda,Ahorros,Retiro,100
2,2024-12-26 23:20:30.945597,Ana Laura Ruiz,Corriente,Deposito recibido,200


La anterior tabla nos muestran los movimientos realizados junto con los depósitos recibidos.

Ahora pedimos que muestre la base de datos de los clientes junto con su saldo actual y es la siguiente:

In [187]:
df_clientes

,Nombre,Tipo de Cuenta,Numero de Cuenta,Saldo,Fecha de Transaccion
0,Juliana Castaneda,Ahorros,0001,200,2024-12-26 23:20:31.717333
1,Ana Laura Ruiz,Corriente,0002,1700,2024-12-26 23:20:31.717333


**VALIDACIONES DE SALDO NEGATIVO**

Ahora comprobaremos que cuando el cliente quiera realizar una transacción que no cumpla con el saldo mínimo admitido, no permitirá realizarla:

In [188]:
# ------------ REGISTRO DE CLIENTES ------------

# Agregamos dos clientes nuevos

cliente3 = Clientes("0003", "Juan Manuel", cuenta_ahorros, 800)
cliente4 = Clientes("0004", "Diana Laura", cuenta_corriente, 100)

clientes.append(cliente3)
clientes.append(cliente4)


# Actualizar y mostrar la información de los clientes y sus movimientos
df_clientes = crear_dataframe_clientes(clientes)
df_clientes                                         # Muestra información de los clientes antes de las transacciones



,Nombre,Tipo de Cuenta,Numero de Cuenta,Saldo,Fecha de Transaccion
0,Juliana Castaneda,Ahorros,0001,200,2024-12-26 23:20:32.253294
1,Ana Laura Ruiz,Corriente,0002,1700,2024-12-26 23:20:32.253294
2,Juan Manuel,Ahorros,0003,800,2024-12-26 23:20:32.253294
3,Diana Laura,Corriente,0004,100,2024-12-26 23:20:32.253294


Podemos observar que el cliente Juan Manuel tiene una cuenta de ahorros y que cuenta con un saldo de $800, entonces no puede retirar más de esa cantidad. De lo contrario, el sistema no lo dejará.

In [189]:
# ------------ RETIROS ------------

cliente3.retirar(850)


ERROR: Fondos insuficientes en una cuenta de ahorros.


##### **III) Implementa un sistema de auditoría que registre en un archivo CSV con todas las transacciones realizadas en el banco, indicando la fecha, el cliente, el tipo de cuenta y la cantidad transferida o retirada.**

In [190]:
df_movimientos

,Fecha,Cliente,Tipo de Cuenta,Tipo de Movimiento,Monto
0,2024-12-26 23:20:30.945597,Juliana Castaneda,Ahorros,Deposito a tercero,200
1,2024-12-26 23:20:31.717333,Juliana Castaneda,Ahorros,Retiro,100
2,2024-12-26 23:20:30.945597,Ana Laura Ruiz,Corriente,Deposito recibido,200


In [191]:
# Transformar el CSV del registro de movimientos y nombrarlo auditoría

df_movimientos.to_csv('auditoria.csv', index=False)